#Retrieving Twitter Wordle scores

The function of this script is to retrieve Wordle scores from Twitter, assess them for their validity, and upload them to a SQL server hosted on my computer.

In [2]:
import tweepy
import pandas as pd
import numpy as np
import pyodbc
import secret

##Connecting to the Twitter API and retrieving search results

In order to retrieve a list of Twitter search results, I first have to connect to the server. In this case, I've used a Bearer Token specific to my Twitter account.

After completing a Wordle, the game allows you to share a text of your game's results, which proud (or angry) players do to Twitter quite frequently. To find these scores, I specifically searched for the word "Wordle" and number "6." As seen below, each game report starts with "Wordle" followed by the day and score out of six. Since the day and score change day-to-day and by player, I limited my search results to the constants within the score reports

**Wordle** 345 3/**6**

🟩⬛🟩⬛🟨<br>
⬛⬛⬛⬛⬛<br>
🟩🟩🟩🟩🟩<br>

*An example board after completing the Wordle and the terms searched for*

In [3]:
#accessing the Twitter API
client = tweepy.Client(bearer_token = secret.BEARER)

In [4]:
#specifying query for searching Twitter
query = 'Wordle 6 -is:retweet'

#searching tweets per the query
tweets = client.search_recent_tweets(query=query,max_results= 100, tweet_fields=['author_id', 'created_at','geo'])

##Loading Tweet information into a data frame

In [5]:
data = pd.DataFrame(data = [tweet.id for tweet in tweets.data], columns =['ID'])
data['Author'] = np.array([tweet.author_id for tweet in tweets.data])

data.head(10)

,ID,Author
0,1536459683212443649,1319940115494543361
1,1536459681723465729,9724842
2,1536459659133210624,154500670
3,1536459658508087301,1525144791683584000
4,1536459655810981888,348244925
5,1536459655752318976,1093068939645644800
6,1536459648739717121,2458786034
7,1536459646407528450,1005101136
8,1536459638270738432,117980706
9,1536459632335794176,2312617050


For loading the scores, days, and if hard mode was activated, I found and seperated these values based on common locations and layouts of Wordle scores. By finding the location of the "/" I was able to locate the score, day, and hard mode based on the relative location. By finding these, I then added them to a list and then updated the Pandas DataFrame accordingly.

One common issue is the abundance of Wordle clones. For varying subject matters and languages, games exist to correctly guess the NBA player, BTS lyric, or five-letter Spanish word. Most of these variants follow a different format and are filtered out, but commonly appear in the search results, so 100 records are not uploaded to the database every time the script is ran. 

In [6]:
score_arr = []
day_arr = []
isHard_arr = []

for tweet in tweets.data:
    score = tweet.text.find("/")-1

    #Determining the score for the tweet and adding it to the score array
    if tweet.text[score].isnumeric():
        score_arr.append(tweet.text[score])
    else:
        score_arr.append(0)

    temp_day = tweet.text.partition("Wordle ")
    day = temp_day[2][0:4]

    #Finding the day associated with the tweet and marking it if the format is not correct
    if(day[0:3].isnumeric() and day[3] == " "):
        day_arr.append(day[0:3])
    else:
        day_arr.append('NaN')

    #Determining if the user is playing on hard mode
    if(tweet.text[score+3] == '*'):
        isHard_arr.append(1)
    else:
        isHard_arr.append(0)

In [7]:
data['WordleDay'] = day_arr
data['Score'] = score_arr
data['HardMode'] = isHard_arr

In [8]:
#Removing any scores which did not meet the criteria for a Wordle but appeared in the search results
clean_data = data[data['WordleDay'].str.contains("NaN") == False]

clean_data.head(10)

,ID,Author,WordleDay,Score,HardMode
0,1536459683212443649,1319940115494543361,359,4,0
1,1536459681723465729,9724842,359,3,0
2,1536459659133210624,154500670,360,1,0
3,1536459658508087301,1525144791683584000,359,5,0
4,1536459655810981888,348244925,360,4,0
6,1536459648739717121,2458786034,360,4,0
8,1536459638270738432,117980706,360,2,1
9,1536459632335794176,2312617050,360,2,0
10,1536459631467409409,15134354,360,5,0
13,1536459627063304192,3697184955,359,4,0


##Database upload

After properly cleaning, we are left with a DataFrame ready to upload! I created a SQL database on my desktop computer and upload the values there. I've set the script to run every 30 minutes to pull down new scores, quickly aggregating a day's scores and getting a large sample size! Next, my job is to sit back and wait for scores to come in until I can analyze the data.

In [9]:
#uploading dataframe to the databse
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=AUSTIN-PC\SQLEXPRESS;'
                      'Database=WordleAnalysis;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

for index, row in clean_data.iterrows():
    cursor.execute("INSERT INTO TwitterWordle (ID,Author,WordleDay,Score,HardMode) values(?,?,?,?,?)", row.ID, row.Author, row.WordleDay, row.Score, row.HardMode)
conn.commit()
cursor.close()